In [1]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml import automl
from azure.ai.ml.entities import Data, Datastore
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
import numpy as np
import pandas as pd
import mltable
import fsspec

In [2]:
client = MLClient.from_config(credential=DefaultAzureCredential(), path=".")
ws = client.workspaces.get(name=client.workspace_name)
print(ws.name)

Found the config file in: /mnt/batch/tasks/shared/LS_root/mounts/clusters/e4s-4c32g/code/.azureml/config.json


ml_test


In [3]:
data_asset = client.data.get("nms_itg_240605", version="1")
df = mltable.load(f'azureml:/{data_asset.id}').to_pandas_dataframe()
df.head()

,five_minute_interval,RU,RRC,MAX_UE,AirMAC,PRB,Watt
0,2024-04-28 16:50:00,NUKG33951S,34.0,8.0,NaN,1.51,NaN
1,2024-04-28 16:50:00,NUKG33630S,121.0,12.0,NaN,2.74,NaN
2,2024-04-28 16:50:00,NUKG33241S,72.0,13.0,NaN,1.87,NaN
3,2024-04-28 16:50:00,NUKG33190S,51.0,11.0,NaN,3.13,NaN
4,2024-04-28 16:50:00,NUKG39569S,45.0,19.0,NaN,4.62,NaN


In [4]:
# Training MLTable defined locally, with local data to be uploaded
data_input = Input(
    type=AssetTypes.MLTABLE, 
    path="./data-small"
)

In [7]:
# note that the below is a code snippet -- you might have to modify the variable values to run it successfully
forecasting_job = automl.forecasting(
    compute="hwtest",
    experiment_name="nms-aut0mlModel_WholeData",
    training_data=data_input,
    target_column_name="Watt",
    primary_metric="normalized_root_mean_squared_error",
    n_cross_validations="auto",    
)

# Limits are all optional
forecasting_job.set_limits(
    timeout_minutes=360,
    trial_timeout_minutes=90,
    max_concurrent_trials=2,
)

# Forecasting specific configuration
forecasting_job.set_forecast_settings(
    time_column_name="five_minute_interval",
    forecast_horizon=12,
    time_series_id_column_names='RU',
    target_lags='auto', 
    target_rolling_window_size='auto',
    feature_lags='auto',
    frequency='H',
    target_aggregate_function='sum'
)

forecasting_job.set_training(
    allowed_training_algorithms=["ExponentialSmoothing", "Prophet"]
)

In [8]:
returned_job = client.jobs.create_or_update(
    forecasting_job
)

print(f"Created job: {returned_job}")

# Get a URL for the job in the AML studio user interface
returned_job.services["Studio"].endpoint

Uploading data-small (76.58 MBs): 100%|██████████| 76583593/76583593 [00:00<00:00, 111490822.37it/s]




Created job: compute: azureml:hwtest
creation_context:
  created_at: '2024-06-14T12:33:34.544571+00:00'
  created_by: user07
  created_by_type: User
display_name: olive_frog_2gd62v1q32
experiment_name: nms-aut0mlModel_WholeData
forecasting:
  feature_lags: auto
  forecast_horizon: 12
  frequency: H
  seasonality: auto
  short_series_handling_config: auto
  target_aggregate_function: sum
  target_lags: auto
  target_rolling_window_size: auto
  time_column_name: five_minute_interval
  time_series_id_column_names: '[''RU'']'
  use_stl: none
id: azureml:/subscriptions/05f2e45e-7783-4a5b-ad4d-c498610f67fb/resourceGroups/rgfabricpoc/providers/Microsoft.MachineLearningServices/workspaces/ml_test/jobs/olive_frog_2gd62v1q32
limits:
  enable_early_termination: true
  max_concurrent_trials: 2
  max_cores_per_trial: -1
  max_nodes: 1
  max_trials: 1000
  timeout_minutes: 360
  trial_timeout_minutes: 90
log_verbosity: info
n_cross_validations: auto
name: olive_frog_2gd62v1q32
outputs: {}
primary_me

'https://ml.azure.com/runs/olive_frog_2gd62v1q32?wsid=/subscriptions/05f2e45e-7783-4a5b-ad4d-c498610f67fb/resourcegroups/rgfabricpoc/workspaces/ml_test&tid=769322b2-fb02-4625-b63d-10cedd4cd7de'